# Weekly Project 5
## Global Registration implementation.
## Task 1
Today your project is to implement a global registration algorithm.

It should be able to roughly align two pointclouds.
1. Implement global registration
2. Can you fit **r1.pcd** and **r2.pcd**?
3. Can you fit **car1.ply** and **car2.ply**?
These are in the *global_registration* folder


In [328]:
import open3d as o3d
import numpy as np
import copy
from scipy.spatial import procrustes
import scipy

In [329]:
# helper function for drawing if you want it to be more clear which is which set recolor=True
def draw_registrations(source, target, transformation = None, recolor = False):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    if(recolor):
        source_temp.paint_uniform_color([1, .1, 0])
        target_temp.paint_uniform_color([0, .1, 1])
    if(transformation is not None):
        source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

In [456]:
source = o3d.io.read_point_cloud("global_registration/r1.pcd")
target = o3d.io.read_point_cloud("global_registration/r2.pcd")
voxel_size = 0.075
# draw_registrations(source, target)

PointCloud with 137833 points.

In [462]:
mesh = o3d.io.read_triangle_mesh("global_registration/car1.ply")
pcd1 = o3d.geometry.PointCloud()
pcd1.points = o3d.utility.Vector3dVector(np.array(mesh.vertices))
draw_registrations(pcd1,pcd1)

[Open3D WARNING] geometry::TriangleMesh appears to be a geometry::PointCloud (only contains vertices, but no triangles).


### Features

In [343]:
source = source.voxel_down_sample(voxel_size)
source.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2.0,
                                            max_nn=30))
source_fpfh_features = o3d.pipelines.registration.compute_fpfh_feature(source,
    o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 5.0,max_nn=100))

target = target.voxel_down_sample(voxel_size)
target.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2.0,
                                            max_nn=30))
target_fpfh_features = o3d.pipelines.registration.compute_fpfh_feature(target,
    o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 5.0, max_nn=100))

In [438]:
target_fpfh_features.data.shape, source_fpfh_features.data.shape, target.dimension, source.dimension

((33, 1615),
 (33, 2154),
 <bound method PyCapsule.dimension of PointCloud with 1615 points.>,
 <bound method PyCapsule.dimension of PointCloud with 2154 points.>)

In [345]:
target_points = np.asarray(target.points)
target_centroid = np.mean(target_points, axis=0)
source_points = np.asarray(source.points)
source_centroid = np.mean(source_points, axis=0)

In [411]:
# data = np.append(target_fpfh_features.data.T,source_fpfh_features.data.T,axis=0)
kd1 = scipy.spatial.KDTree(target_fpfh_features.data.T)
kd2 = scipy.spatial.KDTree(source_fpfh_features.data.T)
indexes = kd1.query_ball_tree(kd2, r=13)
# target => source
matches = {}
for i in range(len(indexes)):
    if indexes[i]:
        matches[i] = indexes[i][0]
n_matched = len(matches)

In [432]:
target_random_sample = np.random.choice(list(matches.keys()), (3,), replace=False)
source_random_sample = [matches[target_random_sample[x]] for x in range(3)]
target_random_sample, source_random_sample

(array([  63, 1597,  370]), [15, 104, 849])

In [448]:
match_targets = target_points[list(matches.keys())]
match_sources = source_points[list(matches.values())]
match_targets.shape == match_sources.shape

True

In [435]:
target_matched_xyz = target_points[target_random_sample] - target_centroid
source_matched_xyz = source_points[source_random_sample] - source_centroid
target_matched_xyz,source_matched_xyz

(array([[-0.1864865 , -0.81110513,  0.02239685],
        [ 0.62712275,  0.64703903, -0.45486944],
        [-0.17946324, -0.67165316,  0.06464461]]),
 array([[ 0.25414949,  0.15883807, -0.7056722 ],
        [-0.84843637, -0.36647425,  0.50610426],
        [ 0.11138434,  0.6090613 , -0.55680207]]))

In [439]:
(estimated_rotation,rmsd) = scipy.spatial.transform.Rotation.align_vectors(target_matched_xyz, source_matched_xyz)
estimated_rotation,rmsd

(<scipy.spatial.transform._rotation.Rotation at 0x1943a5fd6c0>,
 0.3750123451566961)

In [453]:
# match_targets.shape == match_sources.shape
np.linalg.norm(estimated_rotation.apply(match_sources) - match_targets)

33.338443509045206

In [350]:
# n_matched = min(target_fpfh_features.data.shape[1], source_fpfh_features.data.shape[1])

# n_target_f = target_fpfh_features.data.shape[1]
# n_source_f = source_fpfh_features.data.shape[1]

# p_over_norm = False

# min_disparity = np.inf
# min_target_sample = np.zeros((3,))
# min_source_sample = np.zeros((3,))
# # for i in range(10000):
# target_random_sample = np.random.choice(np.arange(n_target_f), (3,), replace=False)
# source_random_sample = np.random.choice(np.arange(n_source_f), (3,), replace=False)
# if p_over_norm:
#     mtx1, mtx2, disparity = procrustes(
#         target_fpfh_features.data[:,target_random_sample], 
#         source_fpfh_features.data[:,source_random_sample])
# else:
#     disparity = np.linalg.norm(
#         target_fpfh_features.data[:,target_random_sample] - 
#         source_fpfh_features.data[:,source_random_sample])
# if disparity < min_disparity:
#     min_disparity = disparity
#     min_target_sample = target_random_sample
#     min_source_sample = source_random_sample

# print(min_disparity, min_target_sample, min_source_sample)

276.62869752545635 [1110 1091  483] [ 730 1097  380]


In [336]:
target_matched_xyz = target_points[min_target_sample] - target_centroid
source_matched_xyz = source_points[min_source_sample] - source_centroid

In [474]:
x = np.array([[0, 2], [1, 1], [2, 0]]).T
x.shape, target_points[target_random_sample], 

np.append(target_points[target_random_sample], source_points[source_random_sample],axis=0).shape, x.shape
source_points.shape

(2154, 3)

In [337]:
# Cpq = target_matched_xyz @ source_matched_xyz.T
# U,S,V = np.linalg.svd(Cpq)
# U @ V

In [338]:
(estimated_rotation,rmsd) = scipy.spatial.transform.Rotation.align_vectors(target_matched_xyz, source_matched_xyz)
estimated_rotation,rmsd

(<scipy.spatial.transform._rotation.Rotation at 0x1943ac42990>,
 1.0179907252439597)

In [353]:

estimated_rotation.apply(source_matched_xyz),target_matched_xyz


(array([[-0.48978131, -0.64845683,  0.13119908],
        [-0.22432958, -0.35277494,  0.03728706],
        [-0.18791448,  0.48169804, -0.12840193]]),
 array([[-0.67780466, -0.27464229,  0.02694767],
        [ 0.06418354,  0.17066639, -0.02039324],
        [-0.88154976,  0.55336459, -0.18775678]]))

In [339]:
target_centered = target_points - target_centroid
source_centered = source_points - source_centroid
source_centered_transformed = estimated_rotation.apply(source_centered)

In [352]:
disparity = np.linalg.norm(source_centered_transformed - target_centered)

ValueError: operands could not be broadcast together with shapes (3237,3) (2446,3) 

In [340]:
# pcd1 = o3d.geometry.PointCloud()
# pcd1.points = o3d.utility.Vector3dVector(source_centered_transformed)
# pcd2 = o3d.geometry.PointCloud()
# pcd2.points = o3d.utility.Vector3dVector(target_centered)
# draw_registrations(pcd1, pcd2, recolor=True)

In [341]:
# h = mapping_points.T @ true_points
# u, s, vt = np.linalg.svd(h)
# v = vt.T
# d = np.linalg.det(v @ u.T)
# e = np.array([[1, 0, 0], [0, 1, 0], [0, 0, d]])
# r = v @ e @ u.T

## Task 2 (Challange)
Challanges attempt either or both:
- Implement local registration.

- Attempt to reconstruct the car from the images in *car_challange* folder.

You can use the exercises from monday as a starting point.